참고 //
OpenAI ChatGPT API를 활용해 추천 시스템 구현하기(feat. HuggingFace), 이수진의 블로그
https://lsjsj92.tistory.com/657

lsjsj92/recommender_system_with_Python
https://github.com/lsjsj92/recommender_system_with_Python/blob/master/009_chatgpt_recsys.ipynb

HuggingFace model
snunlp/KR-SBERT-V40K-klueNLI-augSTS
https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS/tree/main

DataSet
TourAPI, 한국관광공사
https://api.visitkorea.or.kr/#/

In [18]:
import os

import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import openai
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")
# cur_os = sys.platform
# model_path = f"D:/github" if cur_os.startswith('win') else None

In [4]:
metadata = pd.read_csv('./tourInfo_copy.csv')
print(metadata.shape)
metadata.head()

(2004, 21)


,Unnamed: 0,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,...,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode
0,0,인천광역시 중구 연안부두로 61,NaN,2,NaN,A05,A0502,A05020100,597467,39,...,http://tong.visitkorea.or.kr/cms/resource/74/1...,http://tong.visitkorea.or.kr/cms/resource/74/1...,126.599864,37.453488,6.0,2.022120e+13,10,032-882-8505,가덕도횟집,22348
1,1,인천광역시 계양구 경명대로 977,NaN,2,NaN,A05,A0502,A05020100,2876098,39,...,http://tong.visitkorea.or.kr/cms/resource/96/2...,http://tong.visitkorea.or.kr/cms/resource/96/2...,126.715515,37.544160,6.0,2.022100e+13,2,NaN,가든파티 계양점,21042
2,2,인천광역시 남동구 오봉동로4번길 9,(도림동),2,NaN,A05,A0502,A05020100,2768130,39,...,NaN,NaN,126.729889,37.416801,6.0,2.022120e+13,4,NaN,가리비칼국수,21656
3,3,인천광역시 미추홀구 소성로326번길 4,NaN,2,NaN,A05,A0502,A05020100,135696,39,...,http://tong.visitkorea.or.kr/cms/resource/54/1...,http://tong.visitkorea.or.kr/cms/resource/54/1...,126.683246,37.437397,6.0,2.022100e+13,3,032-426-7270,가마솥손두부,22238
4,4,인천광역시 부평구 평천로 394,NaN,2,NaN,A05,A0502,A05020400,600680,39,...,http://tong.visitkorea.or.kr/cms/resource/71/1...,http://tong.visitkorea.or.kr/cms/resource/71/1...,126.734017,37.517217,6.0,2.022080e+13,6,032-512-8552,가보,21323


In [5]:
metadata = metadata[metadata['contenttypeid'].isin([12, 14])]
metadata.head()

,Unnamed: 0,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,...,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode
11,11,인천광역시 연수구 청량로102번길 40-9,NaN,2,0.0,A02,A0206,A02060100,130105,14,...,http://tong.visitkorea.or.kr/cms/resource/75/1...,http://tong.visitkorea.or.kr/cms/resource/75/1...,126.657604,37.418224,6.0,2.022070e+13,8,032-833-4747,가천박물관,21942
16,16,인천광역시 서구 가정로98번길 18,(가좌동),2,0.0,A02,A0202,A02020700,2767573,12,...,http://tong.visitkorea.or.kr/cms/resource/58/2...,http://tong.visitkorea.or.kr/cms/resource/58/2...,126.673620,37.493238,6.0,2.022120e+13,7,NaN,감중공원,22838
17,17,인천광역시 강화군 강화읍 해안동로1366번길 18,NaN,2,0.0,A02,A0201,A02010200,1254680,12,...,http://tong.visitkorea.or.kr/cms/resource/72/1...,http://tong.visitkorea.or.kr/cms/resource/72/1...,126.517114,37.733538,6.0,2.022050e+13,1,NaN,갑곶돈대,23027
18,18,인천광역시 강화군 강화읍 해안동로1366번길 35,NaN,2,0.0,A02,A0201,A02010900,2768133,12,...,http://tong.visitkorea.or.kr/cms/resource/26/2...,http://tong.visitkorea.or.kr/cms/resource/26/2...,126.517494,37.734196,6.0,2.022100e+13,1,NaN,갑곶순교성지,23027
22,22,인천광역시 강화군 강화읍 동문로 119,(강화읍),2,0.0,A01,A0101,A01010500,1619788,12,...,http://tong.visitkorea.or.kr/cms/resource/80/1...,http://tong.visitkorea.or.kr/cms/resource/80/1...,126.494854,37.754146,6.0,2.022060e+13,1,NaN,강화 갑곶리 탱자나무,23022


In [11]:
metadata = metadata[['addr1','addr2','firstimage','firstimage2', 'mapx', 'mapy', 'tel', 'title']]
metadata.head()

,addr1,addr2,firstimage,firstimage2,mapx,mapy,tel,title
11,인천광역시 연수구 청량로102번길 40-9,NaN,http://tong.visitkorea.or.kr/cms/resource/75/1...,http://tong.visitkorea.or.kr/cms/resource/75/1...,126.657604,37.418224,032-833-4747,가천박물관
16,인천광역시 서구 가정로98번길 18,(가좌동),http://tong.visitkorea.or.kr/cms/resource/58/2...,http://tong.visitkorea.or.kr/cms/resource/58/2...,126.673620,37.493238,NaN,감중공원
17,인천광역시 강화군 강화읍 해안동로1366번길 18,NaN,http://tong.visitkorea.or.kr/cms/resource/72/1...,http://tong.visitkorea.or.kr/cms/resource/72/1...,126.517114,37.733538,NaN,갑곶돈대
18,인천광역시 강화군 강화읍 해안동로1366번길 35,NaN,http://tong.visitkorea.or.kr/cms/resource/26/2...,http://tong.visitkorea.or.kr/cms/resource/26/2...,126.517494,37.734196,NaN,갑곶순교성지
22,인천광역시 강화군 강화읍 동문로 119,(강화읍),http://tong.visitkorea.or.kr/cms/resource/80/1...,http://tong.visitkorea.or.kr/cms/resource/80/1...,126.494854,37.754146,NaN,강화 갑곶리 탱자나무


In [6]:
# tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
# model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [7]:
metadata['addr1'] = metadata['addr1'].astype(str)

In [8]:
metadata['addr_em'] = metadata['addr1'].apply(lambda x: model.encode(x))
metadata.head()

,Unnamed: 0,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,...,firstimage2,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,addr_em
11,11,인천광역시 연수구 청량로102번길 40-9,NaN,2,0.0,A02,A0206,A02060100,130105,14,...,http://tong.visitkorea.or.kr/cms/resource/75/1...,126.657604,37.418224,6.0,2.022070e+13,8,032-833-4747,가천박물관,21942,"[-0.6274227, -0.2258779, -0.5644354, 0.1303640..."
16,16,인천광역시 서구 가정로98번길 18,(가좌동),2,0.0,A02,A0202,A02020700,2767573,12,...,http://tong.visitkorea.or.kr/cms/resource/58/2...,126.673620,37.493238,6.0,2.022120e+13,7,NaN,감중공원,22838,"[-0.9867481, -0.0023858547, -0.44634286, 0.378..."
17,17,인천광역시 강화군 강화읍 해안동로1366번길 18,NaN,2,0.0,A02,A0201,A02010200,1254680,12,...,http://tong.visitkorea.or.kr/cms/resource/72/1...,126.517114,37.733538,6.0,2.022050e+13,1,NaN,갑곶돈대,23027,"[-0.33432502, 0.73459154, -1.0436915, 0.563864..."
18,18,인천광역시 강화군 강화읍 해안동로1366번길 35,NaN,2,0.0,A02,A0201,A02010900,2768133,12,...,http://tong.visitkorea.or.kr/cms/resource/26/2...,126.517494,37.734196,6.0,2.022100e+13,1,NaN,갑곶순교성지,23027,"[0.06275128, 0.20978412, -0.5320568, 0.4413536..."
22,22,인천광역시 강화군 강화읍 동문로 119,(강화읍),2,0.0,A01,A0101,A01010500,1619788,12,...,http://tong.visitkorea.or.kr/cms/resource/80/1...,126.494854,37.754146,6.0,2.022060e+13,1,NaN,강화 갑곶리 탱자나무,23022,"[-0.3749388, 0.46581152, -0.56632894, 0.372161..."


In [9]:
metadata.to_csv('./tourInfo_em.csv')

In [10]:
top_k=5

In [25]:
def get_query_sim_top_k(query, model, df, top_k):
    query_encode = model.encode(query)
    df_array = np.array(metadata['addr_em'].tolist())
    df_tensor = torch.tensor(df_array)
    cos_scores = util.pytorch_cos_sim(query_encode, df_tensor)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    return top_results

In [26]:
query = '부평구에 뭐가 있어'
top_results = get_query_sim_top_k(query, model, metadata, top_k)

In [27]:
top_results

torch.return_types.topk(
values=tensor([0.6119, 0.5971, 0.5942, 0.5874, 0.5807]),
indices=tensor([175, 174, 213, 163, 249]))

In [28]:
metadata.iloc[top_results[1].numpy(), :]['title'].values

array(['부평공원', '부평 해물탕거리', '세어도', '백운공원', '신트리공원'], dtype=object)

In [29]:
query = "중구에 뭐가 있어"
top_results = get_query_sim_top_k(query, model, metadata, top_k)
metadata.iloc[top_results[1].numpy(), :]['title'].values

array(['국사봉(인천)', '실미해수욕장', '왕산해수욕장', '재미난 박물관', '영종진공원'], dtype=object)

In [30]:
query = "미추홀구에 갈만한 곳 뭐 있을까"
top_results = get_query_sim_top_k(query, model, metadata, top_k)
metadata.iloc[top_results[1].numpy(), :]['title'].values

array(['인천수봉문화회관', '은율탈춤전수관', '인천통일관', '수봉공원', '주인근린공원'], dtype=object)

In [73]:
def chat_msg(msg):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=msg
    )
    return completion.choices[0].message.content

In [74]:
messages = [
    {"role":"system", "content":"너는 인천 여행 가이드야. 인천 여행 가이드라고 소개하고 모르는 답변이 있으면 고객센터 1234-5678로 연락달라 그래."},
    {"role":"user", "content":"인천에 추천할 만한 여행지 알려줘"}
]

In [75]:
print(chat_msg(messages))

네, 인천에는 다양한 여행지가 있습니다. 몇 가지 추천해드릴게요.

1. 인천 차이나타운
인천 중구 영종시장에 위치한 차이나타운은 중국 문화와 요리를 경험할 수 있는 곳입니다. 중국식 거리음식도 즐길 수 있어요.

2. 인천 차이나타운 문화공원
인천 차이나타운 안에 위치한 이곳은 중국의 전통 건축과 정원을 볼 수 있는 공원입니다.

3. 월미도
월미도는 인천시 남동구에 있는 섬으로 유람선이나 배를 타고 가는 것이 가능하며, 바다에서 자유로운 분위기를 느낄 수 있습니다.

4. 송도 센트럴파크
송도의 대표적인 공원인 센트럴파크는 경치가 좋아 사람들의 인기를 끌고 있습니다. 또한, 송도의 패션문화거리나 인천 아시아드주 경기장을 산책하며 관광할 수 있습니다.

5. 인천대공원
인천의 대표적인 공원으로, 조계사 사찰, 자연사박물관, 수목원 등 다양한 관광지가 있습니다.

이 외에도 인천 타임스퀘어, 인천국제공항 스카이라이프, 해운대 유람선 터미널 등 다양한 관광지가 있습니다. 해당 정보가 도움이 되셨길 바랍니다.
